# Vectorize Images with Siamese Network

In [1]:
import os
import glob

import numpy as np
from PIL import Image
import pandas as pd

import torch
import torch.nn as nn
import torchvision

#### Parameters

In [2]:
logos_path = os.path.join(os.environ['DATASETS'], 'logos', 'logos', 'all_crops')
exemplars_path = os.path.join(os.environ['DATASETS'], 'logos', 'exemplars')
model_filename = os.path.join(os.environ['HKT'], 'Logos-Recognition',
                              'models', 'siamese_embedding.pth')
img_size = 100
model_out = 345

In [3]:
def calc_nn_feat(img_path, transform, model):
    img = Image.open(img_path).convert('RGB')
    img_t = transform(img)
    features = model(img_t.unsqueeze(0).cuda())
    return features.detach().cpu().numpy().squeeze()

#### Load model

In [4]:
model = torchvision.models.resnet18(pretrained=False)
model.fc = nn.Linear(model.fc.in_features, model_out)
checkpoint = torch.load(model_filename)
model.load_state_dict(checkpoint['state_dict'])
model.eval()
model.cuda()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [5]:
# Define the transform:
transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((img_size, img_size)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465),
                                     (0.2023, 0.1994, 0.2010)),
])

In [6]:
def clean_name(name):
    '''
    >> ' '.join(sorted(set(''.join(list(set(brands))))))
    >> "& ' + - 1 2 3 4 ? a b c d e f g h i j kl m n
        o p q r s t u v w x y z \udcbc \udcc3 \udcfc"
    '''
    filename = name.split('/')[-1]
    brand = filename.split('_')[0]
    return brand.encode('ascii', 'replace').decode()

In [7]:
def vectorize_and_save(path, model_out, transform, model, save_name=None):
    filenames = glob.glob(path + '/*.jpg')
    
    # Extract vectors:
    vectors = np.zeros((len(filenames), model_out))
    for idx, img_path in enumerate(filenames):
        vectors[idx, :] = calc_nn_feat(img_path, transform, model)

    vectors_list = [v for v in vectors]
    brands = [clean_name(p) for p in filenames]
    logos_df = pd.DataFrame({'brand': brands, 'img_vec': vectors_list})

    # Save data:
    logos_df.to_pickle(path + '{}.zip'.format(save_name))

#### Process logos

In [8]:
vectorize_and_save(logos_path, model_out, transform, model)

#### Process exemplars

In [9]:
vectorize_and_save(exemplars_path, model_out, transform, model, '_siamese')